In [1]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from IPython.display import clear_output

import multiprocessing  

from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import Estimator
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.connectors import TorchConnector


In [3]:
# ----- 构建 1D 横场 Ising Hamiltonian -----
def build_ising_hamiltonian(n, J, h):
    paulis = []
    coeffs = []

    # Z_i Z_{i+1} 项
    for i in range(n - 1):
        label = ['I'] * n
        label[i] = 'Z'
        label[i + 1] = 'Z'
        paulis.append(''.join(reversed(label)))  # Qiskit从右往左
        coeffs.append(-J)

    # X_i 项
    for i in range(n):
        label = ['I'] * n
        label[i] = 'X'
        paulis.append(''.join(reversed(label)))
        coeffs.append(-h)

    return SparsePauliOp.from_list(list(zip(paulis, coeffs)))

hamiltonian = build_ising_hamiltonian(n_qubits, J, h)

In [4]:
def create_hadamard_ansatz(n_qubits, reps=6):
    params = ParameterVector('θ', length=3 * n_qubits * reps)
    qc = QuantumCircuit(n_qubits)
    # Hadamard初态
    for i in range(n_qubits):
        qc.h(i)
    # 多层参数化
    for rep in range(reps):
        for i in range(n_qubits):
            qc.ry(params[3 * n_qubits * rep + i], i)
            qc.rx(params[3 * n_qubits * rep + n_qubits + i], i)
            qc.rz(params[3 * n_qubits * rep + 2 * n_qubits + i], i)
        # 全连接纠缠
        for i in range(n_qubits):
            for j in range(i+1, n_qubits):
                qc.cx(i, j)
    return qc, list(params)



# 使用Hadamard+全连接纠缠ansatz
ansatz_circuit, ansatz_params = create_hadamard_ansatz(n_qubits, reps=6)


In [5]:
# ----- 构建 EstimatorQNN -----
estimator = Estimator()
qnn = EstimatorQNN(
    circuit=ansatz_circuit,
    observables=hamiltonian,
    input_params=[],
    weight_params=ansatz_params,
    estimator=estimator
)


/var/folders/d9/_3jh3gcd7m5gl50nsj1mgls80000gn/T/ipykernel_11582/1738038090.py:3: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = EstimatorQNN(


In [6]:
# ----- 用 TorchConnector 包装 QNN -----
qnn_model = TorchConnector(qnn)

In [7]:
# ----- 动态可视化回调函数 -----
objective_func_vals = []
def callback_graph(weights, obj_func_eval):
    clear_output(wait=True)
    objective_func_vals.append(obj_func_eval)
    plt.title("Objective function value against iteration")
    plt.xlabel("Iteration")
    plt.ylabel("Objective function value")
    plt.plot(range(len(objective_func_vals)), objective_func_vals)
    plt.grid(True)
    plt.show()

In [8]:
# 使用 Adagrad 优化器
'''optimizer = torch.optim.Adagrad(qnn_model.parameters(), lr=0.1)  # 可调整学习率'''

'optimizer = torch.optim.Adagrad(qnn_model.parameters(), lr=0.1)  # 可调整学习率'

In [9]:
# 使用随机梯度下降 (SGD) 优化器
'''optimizer = torch.optim.SGD(qnn_model.parameters(), lr=0.01)  # 可根据需要调整学习率'''


'optimizer = torch.optim.SGD(qnn_model.parameters(), lr=0.01)  # 可根据需要调整学习率'

In [10]:
# 调整 Adam 优化器的学习率
optimizer = torch.optim.Adam(qnn_model.parameters(), lr=0.01)  # 调整学习率


In [20]:
# 对N=2到10分别计算基态能量和磁化率
results = []
for n_qubits in range(6, 7):
    J = 1.0
    h = 1.0
    hamiltonian = build_ising_hamiltonian(n_qubits, J, h)
    ansatz_circuit, ansatz_params = create_hadamard_ansatz(n_qubits, reps=6)
    estimator = Estimator()
    qnn = EstimatorQNN(
        circuit=ansatz_circuit,
        observables=hamiltonian,
        input_params=[],
        weight_params=ansatz_params,
        estimator=estimator
    )
    qnn_model = TorchConnector(qnn)
    num_trials = 1  # 尝试次数，可根据需要调整
    best_energy = None
    best_weights = None

    optimizer = torch.optim.AdamW(qnn_model.parameters(), lr=0.01)
    alpha = 0.02  # X方向磁化率权重
    beta = 0.02   # Z方向磁化率权重

    for trial in range(num_trials):
        for param in qnn_model.parameters():
            nn.init.uniform_(param, a=-np.pi, b=np.pi)
        # objective_func_vals = # 可选：如需可视化

        for epoch in range(100): # 迭代次数
            optimizer.zero_grad()
            output = qnn_model()
            # X方向总磁化
            x_obs = SparsePauliOp.from_list([(f"{'X'*n_qubits}", 1.0)])
            mx = estimator.run(circuits=ansatz_circuit, observables=x_obs, parameter_values=[qnn_model.weight.detach().numpy()]).result().values[0]
            # Z方向总磁化
            z_obs = SparsePauliOp.from_list([(f"{'Z'*n_qubits}", 1.0)])
            mz = estimator.run(circuits=ansatz_circuit, observables=z_obs, parameter_values=[qnn_model.weight.detach().numpy()]).result().values[0]
            # 联合优化能量、X磁化率、Z磁化率
            loss = output.mean() + alpha * (1 - abs(mx)) + beta * (1 - abs(mz))
            loss.backward()
            optimizer.step()
            # callback_graph(qnn_model.weight, loss.item())  # 可选：如需可视化

        final_weights = qnn_model.weight.detach().numpy()
        estimator = Estimator()
        E0 = estimator.run(
            circuits=ansatz_circuit,
            observables=hamiltonian,
            parameter_values=[final_weights]
        ).result().values[0]
        if (best_energy is None) or (E0 < best_energy):
            best_energy = E0
            best_weights = final_weights

    # 用最优参数输出物理量
    estimator = Estimator()
    E0 = estimator.run(circuits=ansatz_circuit, observables=hamiltonian, parameter_values=[best_weights]).result().values[0]
    z_obs = SparsePauliOp.from_list([(f"{'I'*i + 'Z' + 'I'*(n_qubits - i - 1)}", 1.0) for i in range(n_qubits)])
    mz = estimator.run(circuits=ansatz_circuit, observables=z_obs, parameter_values=[best_weights]).result().values[0] / n_qubits
    x_obs = SparsePauliOp.from_list([(f"{'I'*i + 'X' + 'I'*(n_qubits - i - 1)}", 1.0) for i in range(n_qubits)])
    mx = estimator.run(circuits=ansatz_circuit, observables=x_obs, parameter_values=[best_weights]).result().values[0] / n_qubits
    results.append((n_qubits, E0, mz, mx))
    print(f"N={n_qubits}, E0={E0:.6f}, <Z>={mz:.6f}, <X>={mx:.6f}")

/var/folders/d9/_3jh3gcd7m5gl50nsj1mgls80000gn/T/ipykernel_11582/3135510036.py:9: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = EstimatorQNN(


N=6, E0=-6.499263, <Z>=0.375692, <X>=0.700050


In [15]:
# 汇总输出
print("\nSummary for N=2 to 10:")
for n, E0, mz, mx in results:
    print(f"N={n}: E0={E0:.6f}, <Z>={mz:.6f}, <X>={mx:.6f}")


Summary for N=2 to 10:
N=10: E0=-12.131174, <Z>=0.484979, <X>=0.744281
